# Visualizing Years & Years with simple Data Analysis and NLP tools

https://github.com/rhnvrm/lyric-api
https://www.azlyrics.com/y/yearsyears.html

## Import Data

Some of the musics don't have their respective lyrics and must be filtered

In [266]:
import pandas as pd
import urllib.parse
raw = pd.read_csv("songlist.txt", header=None, names=['songs'])
lyrics_url = raw.songs.apply(lambda x : 'http://lyric-api.herokuapp.com/api/find/years%20&%20years/' + urllib.parse.quote(x.lower())).values

In [262]:
import asyncio  
import aiohttp
import requests
import concurrent.futures
import json
import time
import re

## Nest_Asyncio is necessary to run loops on jupyter. If running on separated python script it's not needed
## For more see https://markhneedham.com/blog/2019/05/10/jupyter-runtimeerror-this-event-loop-is-already-running/
import nest_asyncio
nest_asyncio.apply()

def repeat_lines_multiplier(text):
    line_list = []
    for line in text.split('\n'):
        match = re.search("(^.*)x ?(\d+).*$", line)
        if match is None:
            line_list.append(line)
        else:
            for repetitions in range(int(match.group(2))):
                line_list.append(match.group(1))
    return '\n'.join(line_list)

async def get_lyrics(lyrics_url):
    with concurrent.futures.ThreadPoolExecutor(max_workers=200) as executor:
        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                requests.get, 
                url
            )
            for url in lyrics_url
        ]
        lyrics_list = []
        for response in await asyncio.gather(*futures):
            if response.status_code != 200:
                lyrics_list.append('')
            elif json.loads(response.text)['lyric'].find("Unfortunately, we are not licensed to display the full lyrics for this song at the moment") > -1:
                lyrics_list.append('')
            else:
                ## Loops through each line to check for a x3 and repeat that line n times
                lyrics_list.append(repeat_lines_multiplier(json.loads(response.text)['lyric']))
    return lyrics_list

start_time = time.time()
loop = asyncio.get_event_loop()
lyrics = loop.run_until_complete(get_lyrics(lyrics_url))
## We cannot close the loop because the notebook itself is using one. Only closes when using script
# loop.close()
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.21679949760437 seconds ---


In [267]:
lyrics = pd.Series(lyrics)
lyrics.name = "lyrics"
raw = pd.concat([raw, lyrics], axis=1)

I know that Play is a song which is only feated by Years & Years. Therefore the URL doesn't find any lyrics. But if we use the correct main artist we get http://lyric-api.herokuapp.com/api/find/jax%20jones/play I can add this one as seen below

In [268]:
play = requests.get('http://lyric-api.herokuapp.com/api/find/jax%20jones/play')
raw.loc[raw.songs == 'Play', 'lyrics'] = repeat_lines_multiplier(json.loads(play.text)['lyric'])

Below we can still see that some of the songs weren't found in the database. And for those we shall disconsider

In [270]:
## As you can see I'm a huge fan of chaining methods in pandas. Since I've read 
## This post https://towardsdatascience.com/the-unreasonable-effectiveness-of-method-chaining-in-pandas-15c2109e3c69
## I've used for most of my analysis and it helped me a lot debugging and making super complex things without any trouble
(
    raw
    .lyrics
    .apply(lambda x : len(x) > 0)
    .value_counts()
)

True     35
False     9
Name: lyrics, dtype: int64

In [271]:
data = raw[(raw
    .lyrics
    .apply(lambda x : len(x) > 0))]

## Frequentist Analysis (i.e. CountVectorizer)

In [272]:
from sklearn.feature_extraction.text import CountVectorizer
# [NQY18]	J. Nothman, H. Qin and R. Yurchak (2018). “Stop Word Lists in Free Open-source Software Packages”. In Proc. Workshop for NLP Open Source Software.
vectorizer = CountVectorizer(stop_words=['to', 'it', 'the', 'and', 'oh', 'that', 'be', 're', 'are', 'for'])
X = vectorizer.fit_transform(data.lyrics)

In [273]:
freq = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names(), index=data.songs.values)
freq

,13,14,21,about,abused,accelerates,accidental,actin,admission,admit,...,wrong,wrote,ya,yeah,year,years,yesterday,you,young,your
Foundation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,4
Real,0,0,0,0,0,0,0,0,0,0,...,2,0,0,1,0,0,0,55,0,0
Shine,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,38,0,4
Take Shelter,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,33,0,2
Worship,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,43,0,8
Eyes Shut,0,0,0,0,0,0,0,0,0,0,...,2,0,0,2,0,0,0,10,0,3
Ties,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,44,0,2
King,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,9,0,5
Desire,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,26,0,5
Gold,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7,0,0


In [274]:
from plotly import graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)

bars = [
    go.Bar(y=freq.sum().nlargest(10).index[::-1], x=freq.sum().nlargest(10).values[::-1], orientation='h')
]
fig = go.Figure(bars)
py.iplot(fig)

In [296]:
freq.sum().nlargest(10).index[::-1]

Index(['like', 'let', 'love', 'your', 'can', 'don', 'my', 'want', 'me', 'you'], dtype='object')

In [302]:
freq_top_words.sort_values(ascending=False)

you     1.000000
me      0.971429
your    0.800000
my      0.771429
can     0.714286
don     0.685714
like    0.628571
love    0.600000
want    0.542857
let     0.428571
dtype: float64

In [310]:
freq_top_words.index

Index(['you', 'me', 'want', 'my', 'don', 'can', 'your', 'love', 'let', 'like'], dtype='object')

In [311]:
freq.sum().nlargest(10).index[::-1]

Index(['like', 'let', 'love', 'your', 'can', 'don', 'my', 'want', 'me', 'you'], dtype='object')

In [317]:
top_words = freq.sum().nlargest(10).index.values
freq_top_words = ((freq.loc[:, top_words] > 0).sum() / len(freq))

from plotly import graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)
x = freq.sum().nlargest(10).index[::-1]
bars = [
    go.Bar(x=x, y=freq.sum().nlargest(10).values[::-1]),
    go.Bar(x=x, y=freq_top_words.values, yaxis='y2')
]
layout = go.Layout(
    yaxis2=dict(
        title="yaxis2 title",
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    ),
)
fig = go.Figure(data=bars, layout=layout)
py.iplot(fig)

In [243]:
from plotly import graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)

heatmap = [
    go.Heatmap(
        z=freq.apply(lambda x : x/x.sum()),
        x=freq.columns,
        y=freq.index
    )
]
fig = go.Figure(heatmap)
py.iplot(fig)

We then preprocess the lyrics turning them to lower case and remoing common puntuaction. Then we split the yrics into a vector of words and calculate the maximum lenght of the corpus we have. In my case the longest lyrics in the corpus is composed of 1159 words. We use this value to pad all the self similarity matrices to output matrices with the same dimensions.

A self similarity matrix is a matrix of correlation across each vector. In our case the input vector is a lyric and each word is treated as a column and row in order. The diagonal of this matrix is always marked as all the words correlate with themselves but if the word appear elsewhere on the lyric the correspondent row and column is also marked. The output matrix is sparse and symmetric and padded in the lower right corner to match the dimensions maxvalue x maxvalue.

The intuition of using self-similarity matrices is that they comprehend the structure and sequence of the lyrics and point out repetitive blocks in the songs. These repetitive blocks in the songs might help determine the genre as pop music easily repeats e refrain many times with a lower variance os words then rap, for example.

In [24]:
preprocessed = df.loc[:, "lyrics"].str.replace('[\(\),:.]', ' ').str.lower().str.split()
maxvalue = preprocessed.apply(lambda x:len(x)).max()
matrices = []
for a in preprocessed:
    a = np.array(a)
    a1 = a
    a2 = a[np.newaxis].T
    diff = maxvalue - len(a1)
    g = np.core.defchararray.equal(a1, a2) #Calcula a maldita matriz
    g = np.pad(g, ((0, diff), (0, diff)), mode='constant') #Adiciona padding para garantir que todos os input tem o mesmo tamanho
    matrices.append(g)

mymatrix = np.array(matrices)
mymatrix = mymatrix.reshape((len(df), maxvalue, maxvalue, 1))

### Label Encoders

We encode the mood labels in the two most popular enconding techniques: One-hot encoding and Label Encoding.

#### One Hot-Encode

In [18]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(df.genre)
encoded_Y = encoder.transform(df.genre)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

#### Label Encoder

In [50]:
#http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.genre)
# le.classes_
labeled_y = le.transform(df.genre) 
# le.inverse_transform(le.transform(df.Mood))

## Neural Networks

In this section we import the keras to be used on the top of Tensorflow, define and train a few models. The architecture of these models is purely empiric. Given the self similarity matrices we input and convolve then with maxpooling layers. The idea of the modelis to observe different hyperparameters on the covolutional layers. The second model is based on the VGG-16.

In [1]:
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Dense, Activation,Conv2D,MaxPooling2D,Flatten,Dropout, Input
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import regularizers

from keras.callbacks import TensorBoard
from time import time

import h5py

from IPython.display import SVG, display
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [2]:
#Based on https://github.com/fchollet/keras/blob/master/examples/babi_rnn.py
#Kernel_size outputs
#https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
#https://keras.io/layers/convolutional/#conv2d
def model1():
    input_layer = Input(shape=(maxvalue, maxvalue, 1),
        name="input_layer",
        dtype='float32')
    network = Conv2D(filters=50, 
        kernel_size=(9,9),
        strides=(2,2),
        kernel_initializer='glorot_uniform',
        name="conv1")(input_layer)
    network = MaxPooling2D(pool_size=(5,5), name="max_pool1")(network)
    network = Conv2D(filters=30, 
        kernel_size=(5,5),
        strides=(2,2),
        kernel_initializer='glorot_uniform',
        name="conv2")(network)
    network = MaxPooling2D(pool_size=(3,3), name="max_pool2")(network)
    network = Conv2D(filters=30, 
        kernel_size=(3,3),
        strides=(1,1),
        kernel_initializer='glorot_uniform',
        name="conv3")(network)
    network = MaxPooling2D(pool_size=(1,1), name="max_pool3")(network)
    network = Flatten()(network)
    network = Dense(100, activation='sigmoid', name="dense")(network)
    network = Dropout(0.2)(network)
    network = Dense(len(df.genre.unique()), activation='softmax', name="mood_output")(network)    
    model = Model(input_layer, network)
    return model

model = model1()
model.summary()

NameError: name 'maxvalue' is not defined

In [17]:
del model

In [28]:
# model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(mymatrix, dummy_y, batch_size=20, epochs=5, validation_split=0.05)
# model.save('./models/model'+str(i+1)+'_'+colorName(j)+'_'+str(epoch)+'.h5')
# epoch += 10

Train on 572 samples, validate on 31 samples
Epoch 1/5
572/572 [==============================] - 10s 17ms/step - loss: 1.4804 - acc: 0.4738 - val_loss: 3.0890 - val_acc: 0.0000e+00
Epoch 2/5
572/572 [==============================] - 8s 15ms/step - loss: 1.2632 - acc: 0.5664 - val_loss: 3.0381 - val_acc: 0.0000e+00
Epoch 3/5
572/572 [==============================] - 8s 15ms/step - loss: 1.2364 - acc: 0.5472 - val_loss: 2.5751 - val_acc: 0.0323
Epoch 4/5
572/572 [==============================] - 8s 15ms/step - loss: 1.2028 - acc: 0.5717 - val_loss: 2.4706 - val_acc: 0.0000e+00
Epoch 5/5
572/572 [==============================] - 8s 15ms/step - loss: 1.1988 - acc: 0.5490 - val_loss: 2.7203 - val_acc: 0.0323


In [35]:
def model2():
    input_layer = Input(shape=(maxvalue, maxvalue, 1),
        name="input_layer",
        dtype='float32')
    network = Conv2D(filters=30, 
        kernel_size=(3,3),
        kernel_initializer='glorot_uniform',
        name="conv_0_1")(input_layer)
    network = Conv2D(filters=30, 
        kernel_size=(3,3),
        kernel_initializer='glorot_uniform',
        name="conv_0_2")(network)
    network = MaxPooling2D(pool_size=(2,2), name="max_pool_0")(network)
    network = Conv2D(filters=30, 
        kernel_size=(3,3),
        kernel_initializer='glorot_uniform',
        name="conv_1_1")(network)
    network = Conv2D(filters=30, 
        kernel_size=(3,3),
        kernel_initializer='glorot_uniform',
        name="conv_1_2")(network)
    network = MaxPooling2D(pool_size=(2,2), name="max_pool_1")(network)
    for i in range(2,7):
        network = Conv2D(filters=30, 
            kernel_size=(3,3),
            kernel_initializer='glorot_uniform',
            name="conv_{}_1".format(i))(network)
        network = Conv2D(filters=30, 
            kernel_size=(3,3),
            kernel_initializer='glorot_uniform',
            name="conv_{}_2".format(i))(network)
        network = Conv2D(filters=30, 
            kernel_size=(3,3),
            kernel_initializer='glorot_uniform',
            name="conv_{}_3".format(i))(network)
        network = MaxPooling2D(pool_size=(2,2), name="max_pool_{}".format(i))(network)
        

    network = Flatten()(network)
    network = Dense(200, activation='sigmoid', name="dense1")(network)
    network = Dropout(0.2)(network)
    network = Dense(200, activation='sigmoid', name="dense2")(network)
    network = Dropout(0.2)(network)
    network = Dense(200, activation='sigmoid', name="dense3")(network)
    network = Dropout(0.2)(network)
    network = Dense(len(df.genre.unique()), activation='softmax', name="mood_output")(network)    
    model = Model(input_layer, network)
    return model
model2 = model2()
# model2.summary()

In [39]:
model2.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# tensorboard = TensorBoard(log_dir="logs/lyricsCorrelation2")
model2.fit(mymatrix, dummy_y, batch_size=3, epochs=5, validation_split=0.05)
# model.save('./models/model'+str(i+1)+'_'+colorName(j)+'_'+str(epoch)+'.h5')
# epoch += 10

Train on 572 samples, validate on 31 samples
Epoch 1/5
572/572 [==============================] - 56s 99ms/step - loss: 0.0943 - acc: 0.4038 - val_loss: 0.1316 - val_acc: 0.0000e+00
Epoch 2/5
572/572 [==============================] - 47s 83ms/step - loss: 0.0944 - acc: 0.4073 - val_loss: 0.1419 - val_acc: 0.0000e+00
Epoch 3/5
572/572 [==============================] - 47s 83ms/step - loss: 0.0930 - acc: 0.4178 - val_loss: 0.1393 - val_acc: 0.0000e+00
Epoch 4/5
572/572 [==============================] - 48s 83ms/step - loss: 0.0916 - acc: 0.4336 - val_loss: 0.1394 - val_acc: 0.0000e+00
Epoch 5/5
572/572 [==============================] - 48s 85ms/step - loss: 0.0913 - acc: 0.4388 - val_loss: 0.1461 - val_acc: 0.0000e+00


It's easy to observe that the model didn't learn much as the accuracy is close to a random guess (given that are 4 available moods) in both models

## word2vec

We then observe the accuracy of the model if instead of using the self-similarity matrices we used a doc2vec model where each lyric is interpreted as a document.

In [51]:
#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb
#https://radimrehurek.com/gensim/models/doc2vec.html
#https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e
import gensim
from gensim.models import Doc2Vec
# model = Doc2Vec(matrices, vector_size=100, window=8, min_count=5, workers=4)
word_tagged = []
for i in df.index:
    word_tagged.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(df.loc[i, "lyrics"]), tags=[df.loc[i, "genre"]]))

model = gensim.models.doc2vec.Doc2Vec(vector_size=20, min_count=2, epochs=5)
model.build_vocab(word_tagged)

In [52]:
%time model.train(word_tagged, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 5.59 s, sys: 85.7 ms, total: 5.68 s
Wall time: 2.25 s


In [53]:
infered_vectors = []
for i in df.index:
    infered_vectors.append(model.infer_vector(df.loc[i, "lyrics"]))

Then the output of the doc2vec is used as input for a fully connected network

In [9]:
#Based on https://github.com/fchollet/keras/blob/master/examples/babi_rnn.py
#Kernel_size outputs
#https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
def model3():
    input_layer = Input(shape=(50, 1, 1),
        name="input_layer",
        dtype='float32')
    network = Flatten()(input_layer)
    network = Dense(200, activation='sigmoid', name="dense1")(network)
    network = Dropout(0.2)(network)
    network = Dense(200, activation='sigmoid', name="dense2")(network)
    network = Dropout(0.2)(network)
    network = Dense(200, activation='sigmoid', name="dense3")(network)
    network = Dropout(0.2)(network)
    network = Dense(len(df.genre.unique()), activation='softmax', name="mood_output")(network)    
    model = Model(input_layer, network)
    return model

model3 = model3()
# model3.summary()

In [10]:
my_infered_vector = np.array(infered_vectors)
my_infered_vector = my_infered_vector.reshape(len(df), 50, 1, 1)

In [54]:
model3.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model3.fit(my_infered_vector, dummy_y, batch_size=500, epochs=100, validation_split=0.05)

ValueError: Input arrays should have the same number of samples as target arrays. Found 765 input samples and 1015 target samples.

Then again the model didn't presented good results as its results are also little better than a random guess

## SVM
We finnaly try to use an SVM to observe its accuracy in our doc2vec vector classification

In [55]:
from sklearn import svm
clf = svm.SVC()
clf.fit(np.array(infered_vectors), labeled_y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(df.genre, le.inverse_transform(clf.predict(infered_vectors)))

/home/marcos/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.38134592680047225

Giving us a final result little better then the networks